In [ ]:
#basic analysis package
import numpy as np
import pandas as pd
from pathlib import Path
import tifffile as tf
from importlib import reload
from util import pil_imread
#cusom packages
import fiducial_alignment_affine as fa_affine

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

def plot_2d_image(img_2d, zmax=1000, animation = True):
    
    if animation == True:   
    #For Plotting 2d image
        #-------------------------------------------
        fig = px.imshow(
            img_2d,
            width=700,
            height=700,
            binary_string=True,
            binary_compression_level=4,
            binary_backend='pil',
            zmax = zmax,
            animation_frame=0
        )
        #-------------------------------------------

        fig.show()
    else:
        #For Plotting 2d image
        #-------------------------------------------
        fig = px.imshow(
            img_2d,
            width=700,
            height=700,
            binary_string=True,
            binary_compression_level=4,
            binary_backend='pil',
            zmax = zmax,
        )
        #-------------------------------------------

        fig.show()

In [ ]:
def plot_2d_locs_on_2d_image(df_locs_2d_1, img_2d, zmax=1000):
    
    #For Plotting 2d image
    #-------------------------------------------
    fig = px.imshow(
        img_2d,
        width=700,
        height=700,
        binary_string=True,
        binary_compression_level=4,
        binary_backend='pil',
        zmax = zmax
    )
    #-------------------------------------------
    
    #For Plotting 2d dots
    #-------------------------------------------
    fig.add_trace(go.Scattergl(
        x=df_locs_2d_1.x,
        y=df_locs_2d_1.y,
        mode='markers',
        marker_symbol='cross',
        marker=dict(
            #maxdisplayed=1000,
            size=4
            ),
        name = "Gaussian"
        )
    )
    #-------------------------------------------
    
    fig.show()

In [ ]:
#get image paths
Pos = 15
bead_src = Path("/groups/CaiLab/personal/Lex/raw/050222_150genes4binding/fiducials")
bead_src = bead_src / f"MMStack_Pos{Pos}.ome.tif"

tiff_src = Path("/groups/CaiLab/personal/Lex/raw/050222_150genes4binding/notebook_pyfiles/dapi_aligned/HybCycle_22")
tiff_src = tiff_src / f"MMStack_Pos{Pos}.ome.tif"

tiff_src_noalign = Path("/groups/CaiLab/personal/Lex/raw/050222_150genes4binding/HybCycle_22")
tiff_src_noalign = tiff_src_noalign / f"MMStack_Pos{Pos}.ome.tif"

In [ ]:
#read in image
beads = pil_imread(str(bead_src), swapaxes=True)
raw = pil_imread((str(tiff_src)), swapaxes=True)

In [ ]:
raw.shape

In [ ]:
beads.shape

In [ ]:
#look at beads
plot_2d_image(beads[0], zmax=2000)

In [ ]:
#look at raw
plot_2d_image(raw[0], zmax=500)

In [ ]:
#check how off
plot_2d_image(np.array([raw[0][0],beads[0][0]]), zmax=5000)

# Test fiducial alignment on one position

In [ ]:
import time
start = time.time()

image,error = fa_affine.fiducial_alignment_single(tiff_src, bead_src, region_size=9, min_distance=5,
                                                  threshold_abs=800, num_peaks=100, max_dist=1.0,ransac_threshold=0.5,
                                                  include_dapi=False,use_ref_coord=True, swapaxes=True, write=False)
print(f"This task took {(time.time()-start)/60} min")

In [ ]:
#look at displacement ([channel, percent change, displacement])
error

In [ ]:
#look at image to make sure transform looks normal
plot_2d_image(image[0], zmax=3000)

In [ ]:
#check how off
plot_2d_image(np.array([beads[0][0],image[0][0]]), zmax=5000)

# Check sbatch

In [ ]:
path_to_files ="/groups/CaiLab/personal/Lex/raw/Linus_10k_cleared_080918_NIH3T3/notebook_pyfiles/dapi_aligned/fiducial_aligned"
fa_affine.plot_error(path_to_files, num_hybcycles = 80, num_channels = 4, savefig = True, by_pos = False)

In [ ]:
path_to_files ="/groups/CaiLab/personal/Lex/raw/Linus_10k_cleared_080918_NIH3T3/notebook_pyfiles/dapi_aligned/fiducial_aligned_gaussian"
fa_affine.plot_error(path_to_files, num_hybcycles = 80, num_channels = 4, savefig = True, by_pos = False)

# Check average fwhm per field of view

In [ ]:
hybs = np.arange(0,23,1)
poss = np.arange(0,64,1)
bad_pos = []
for hyb in hybs:
    for pos in poss:
        #check if file exists
        try:
            src = f"/groups/CaiLab/personal/Lex/raw/050222_150genes4binding/notebook_pyfiles/dapi_aligned/fiducial_aligned/HybCycle_{hyb}/MMStack_Pos{pos}_error.txt"
            error = pd.read_csv(src, sep = " ", header=None)
        except:
            bad_pos.append(pos)

error_list = []
filt_pos = set(poss) - set(bad_pos) 
for hyb in hybs:
    for pos in filt_pos:
        src = f"/groups/CaiLab/personal/Lex/raw/050222_150genes4binding/notebook_pyfiles/dapi_aligned/fiducial_aligned/HybCycle_{hyb}/MMStack_Pos{pos}_error.txt"
        error = pd.read_csv(src, sep = " ", header=None)
        error_list.append(error)

In [ ]:
comb_error = pd.concat(error_list)
comb_error.columns = ["channel", "percent improvement", "fwhm"]

In [ ]:
comb_error.groupby("channel").sem()["fwhm"][0]*100

In [ ]:
comb_error.groupby("channel").mean()["fwhm"][0]*100